<a href="https://colab.research.google.com/github/DavidDasa/Data_Science_project-/blob/main/S%26P_500_Glassdoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Wikipedia Scraping

## Imports

In [ ]:
import os
import pandas as pd
from datetime import date
from tqdm.auto import tqdm
from selenium import webdriver
from dotenv import dotenv_values
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

## Constants

In [ ]:
# Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
# Wikpedia
WIKIPEDIA_REPLACEMENT = {
  "Alliance Data Systems": "Bread Financial",
  "Alphabet Inc Class A": None, # holding company with few to none reviews
  "Alphabet Inc Class C": None, # holding company with few to none reviews
  "American Water Works Company Inc": "American Water",
  "Apache Corporation": "Apache",
  "Booking Holdings Inc": None, # holding company with few to none reviews
  "Charter Communications": "Spectrum",
  "Cisco Systems": "Cisco Systems",
  "Citizens Financial Group": "Citizens",
  "CMS Energy": "Consumers Energy",
  "Cognizant Technology Solutions": "Cognizant Technology Solutions",
  "Concho Resources": "Concho",
  "D. R. Horton": "D.R. Horton",
  "E*Trade": None,  # company does not exist anymore
  "Edison Int'l": "Southern California Edison",
  "Fortive Corp": None, # holding company with few to none reviews
  "Fortune Brands Home & Security": None, # holding company with few to none reviews
  "Grainger (W.W.) Inc.": "Grainger",
  "HollyFrontier Corp": "HollyFrontier Corporation",
  "Howmet Aerospace": "Arconic",
  "J. B. Hunt Transport Services": "J.B. Hunt Transport",
  "JPMorgan Chase": None, # There are no reviews in 2017-2019
  "Kellogg's": "Kellogg Company",
  "Kohl's Corp.": "Kohl's",
  "Loews Corp.": "Loews Hotels & Co",
  "Lowe's Cos.": "Lowe's Home Improvement",
  "National Oilwell Varco Inc.": "National Oilwell Varco",
  "ViacomCBS": "Paramount",
  "Pinnacle West Capital": None, # holding company with few to none reviews
  "PNC Financial Services": "PNC Financial Services Group",
  "PPG Industries": "PPG",
  "Prudential Financial": "Prudential",
  "Truist Financial": None, # There are no reviews in 2017-2019
  "Under Armour Class A": "Under Armour",
  "Under Armour Class C": "Under Armour",
  "United Airlines Holdings": "United Airlines",
  "Waste Management Inc.": "Waste Management Inc.",
  "Williams Cos.": "Williams",
  "Xylem Inc.": "Xylem",
}

## Utils

In [ ]:
# The given Python function takes a Wikipedia table title and converts it into a CSV file name.
# The function removes the "[edit]" part from the title,
# appends the current date to it, and replaces spaces with underscores in the resulting string.
def wikipedia_table_title_to_csv_file_name(title):
    return (title.split("[edit]")[0] + " " + str(date.today()) + ".csv").replace(" ", "_")

In [ ]:
# Selenium
def scrape_tables_by_url(url):
    driver.get(url)
# These lines find the table titles and tables on the webpage using XPath.
# The first line finds the heading elements (h2) that precede the tables with the class name "wikitable".
# The second line finds the table elements with the class name "wikitable".
# The find_elements() method locates multiple elements that match the given XPath expressions.
    table_titles = driver.find_elements(By.XPATH, '//table[contains(@class, "wikitable")]//preceding-sibling::h2')
    tables = driver.find_elements(By.XPATH, '//table[contains(@class, "wikitable")]')
    for table, title in zip(tables, table_titles):
        df = pd.read_html(table.get_attribute('outerHTML'), encoding="utf-8")[0]
#         This line generates the path for the CSV file by combining the "data/" directory path with
#         the modified table title obtained using the wikipedia_table_title_to_csv_file_name() function.
#         The os.makedirs() function is called to create the directory to store the CSV file.
#         However, there is an issue with this line as mentioned before. It should be modified to os.
#         makedirs(os.path.dirname(csv_path), exist_ok=True) to create the parent directory.
        csv_path = f"data/{wikipedia_table_title_to_csv_file_name(title.text)}"
        os.makedirs(csv_path)
        df.to_csv(csv_path, index=False, encoding="utf-8")

In [ ]:
# Selenium
def scrape_companies_table_by_url(url, replacements=None):
  # Open the provided URL in the browser
  driver.get(url)

  # Find the table containing the company information
  table = driver.find_element(By.XPATH, '//table[contains(@class, "wikitable")][1]')

  # Read the table into a DataFrame using pandas
  df = pd.read_html(table.get_attribute('outerHTML'), encoding="utf-8")[0]

  # Find the links to the company Wikipedia pages
  company_links = driver.find_elements(By.XPATH, f'//table[contains(@class, "wikitable")][1]/tbody/tr/td[2]/a')

  # Add a new column to the DataFrame to store the Wikipedia URLs
  df["Wikipedia Company Page URL"] = [link.get_attribute("href") for link in company_links]

  # Add a new column to the DataFrame to store the company names
  df["Company Name"] = ""

  # Create a new DataFrame to store the scraped data
  new_df = pd.DataFrame(columns=df.columns)

  # Iterate over each row in the DataFrame
  for i in tqdm(range(len(df))):
    security = df.loc[i, "Security"]

    # Check if replacements are provided and if the security is in the replacements dictionary
    if replacements and (security in replacements):
      company_name = replacements[security]

      # If the replacement value is a list, iterate over each name and add it to the new DataFrame
      if isinstance(company_name, list):
        for name in company_name:
          new_df.loc[len(new_df)] = df.loc[i]
          new_df.loc[len(new_df)-1, "Company Name"] = name
      # If the replacement value is a string, add it to the new DataFrame
      elif isinstance(company_name, str):
        new_df.loc[len(new_df)] = df.loc[i]
        new_df.loc[len(new_df)-1, "Company Name"] = company_name
    else:
      # If no replacement is found, scrape the company name from the Wikipedia page
      driver.get(df.loc[i, "Wikipedia Company Page URL"])
      page_title = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//h1[contains(@id, "firstHeading")]')))
      company_name = page_title.text.split("(")[0].strip()
      new_df.loc[len(new_df)] = df.loc[i]
      new_df.loc[len(new_df)-1, "Company Name"] = company_name

  # Drop duplicates based on company name, sort by company name, and reset the index
  return new_df.drop_duplicates("Company Name").sort_values("Company Name").reset_index(drop=True)


In [ ]:
def get_companies_table_and_save_to_db(url, replacements=None):
  df = scrape_companies_table_by_url(url, replacements=replacements)
  df.to_csv("s&p500_components_2020.csv", index=False)
  # ids = insert_companies_data_to_db(df)
  # return ids

## Run

In [ ]:
# Wikipedia updated list of S&P500 components
# URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Wikipedia list of S&P500 components as of 1.1.2020
WIKIPEDIA_URL = "https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=933578501"

# get_companies_table_and_save_to_db(WIKIPEDIA_URL, replacements=WIKIPEDIA_REPLACEMENT)

# Glassdoor Scraping

## Imports

In [ ]:
import re
import os
import pandas as pd
from glob import glob
from time import sleep
from datetime import date
from tqdm.auto import tqdm
from selenium import webdriver
from dotenv import dotenv_values
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

## Constants

In [ ]:
# Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
# Selenium
# זמן המתנה 10 שניות בין לחיצות
WAIT_TIME = 10
# 5 נסיונות לפני שקורס
REPEAT_TIMES = 5

# 3. **STARS_CLASSES_DICT**: This dictionary maps CSS classes used in Glassdoor reviews to their corresponding star ratings.
# Each key-value pair represents a CSS class and its associated star rating.
# For example, if a review has a CSS class of 'css-xd4dom', it corresponds to a star rating of 1.


# Glassdoor
STARS_CLASSES_DICT = {
    'css-xd4dom': 1,
    'css-18v8tui': 2,
    'css-vl2edp': 3,
    'css-1nuumx7': 4,
    'css-s88v13': 5
}
# 4. **V_X_DICT**: This dictionary maps CSS classes used in Glassdoor reviews to their corresponding employee experience levels.
# Each key-value pair represents a CSS class and its associated experience level.
# For example, if a review has a CSS class of 'css-hcqxoa', it corresponds to a high experience level.
V_X_DICT = {
    'css-hcqxoa': 'high',
    'css-1h93d4v': 'middle',
    'css-1kiw93k': 'low',
    'css-10xv9lv': None
}

## Utils

In [ ]:
def login_to_glassdoor():
    try:
        # Check if the search input field is present, indicating that the user is already logged in.
        # If so, return immediately.
        elem = WebDriverWait(driver, WAIT_TIME).until(EC.presence_of_element_located((By.XPATH, '//input[contains(@id, "sc.keyword")]')))
        return
    except TimeoutException:
        # If the search input field is not present (indicating the user is not logged in), continue with the login process.
        pass

    # URL for the Glassdoor login page
    url = "https://www.glassdoor.com/profile/login_input.htm"

    # Maximize the window and navigate to the login page
    driver.maximize_window()
    driver.get(url)
    sleep(WAIT_TIME)

    # Find the username input field and fill it with the Glassdoor username from the .env file
    username_input = driver.find_element(By.XPATH, '//input[contains(@id, "inlineUserEmail")]')
    username_input.send_keys(dotenv_values('.env')['GLASSDOOR_USERNAME'])
    sleep(1)

    # Find the password input field and fill it with the Glassdoor password from the .env file
    password_input = driver.find_element(By.XPATH, '//input[contains(@id, "inlineUserPassword")]')
    password_input.send_keys(dotenv_values('.env')['GLASSDOOR_PASSWORD'])
    sleep(1)

    # Find the login submit button and click it to log in
    login_submit_button = driver.find_element(By.XPATH, '//button[contains(@name, "submit")]')
    login_submit_button.click()
    sleep(WAIT_TIME)


In [ ]:
# Initialize the WebDriver (ChromeDriver in this example)
driver = webdriver.Chrome()

# Function to fetch company metadata from Glassdoor
def fetch_company_metadata(company_name: str):
    # Encode the company name for the URL
    company_name_for_url = company_name.replace("&", "%26").replace(" ", "%20").replace("'", "%27").replace(",", "%2C")

    # Navigate to the Glassdoor search results page for the company
    driver.get(f'https://www.glassdoor.com/Search/results.htm?keyword={company_name_for_url}')

    # Wait for the page to load (you can adjust the delay if needed)
    time.sleep(2)

    # Find the company tile link element
    company_tile_link = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//a[contains(@data-test, "company-tile")]')))

    # Get the company's Glassdoor page URL from the link element
    company_glassdoor_page_url = company_tile_link.get_attribute("href")

    try:
        # Find the company tile image element
        company_tile_img = driver.find_element(By.XPATH, '//a[contains(@data-test, "company-tile")]/div[1]/img')

        # Get the company's Glassdoor logo URL from the image element
        company_glassdoor_logo_url = company_tile_img.get_attribute("src")
    except NoSuchElementException:
        # If logo image element not found, assign an empty string to the logo URL
        company_glassdoor_logo_url = ""

    # Find the company tile rating element and extract the overall rating
    company_tile_rating = driver.find_element(By.XPATH, '//a[contains(@data-test, "company-tile")]/div[1]//strong')
    company_glassdoor_overall_rating = company_tile_rating.text.split(" ")[0].strip()

    # Find the company tile title element and extract the company title
    company_tile_title = driver.find_element(By.XPATH, '//a[contains(@data-test, "company-tile")]/div[2]/h3')
    company_glassdoor_title = company_tile_title.text.strip()

    # Find the company tile subtitle element and split it into industry and number of employees
    company_tile_sub_title = driver.find_element(By.XPATH, '//a[contains(@data-test, "company-tile")]/div[2]/div[1]')
    company_tile_sub_title_list = company_tile_sub_title.text.split("\n")

    # Determine the industry and number of employees based on the subtitle
    if len(company_tile_sub_title_list) == 2:
        company_industry = company_tile_sub_title_list[0].strip()
        company_number_of_employees = company_tile_sub_title_list[1].strip()
    elif len(company_tile_sub_title_list) == 1:
        if "Employees" in company_tile_sub_title_list[0].strip():
            company_industry = ""
            company_number_of_employees = company_tile_sub_title_list[0].strip()
        else:
            company_industry = company_tile_sub_title_list[0].strip()
            company_number_of_employees = ""
    else:
        print("Different length of subtitle at", company_name)
        print(company_tile_sub_title_list)
        company_industry = ""
        company_number_of_employees = ""

    # Find the company tile location element and extract the headquarters location
    company_tile_location = driver.find_element(By.XPATH, '//a[contains(@data-test, "company-tile")]/div[2]/div[2]')
    company_headquarters_location = company_tile_location.text.strip()

    # Find the company tile reviews element and extract the number of reviews
    company_tile_reviews = driver.find_element(By.XPATH, '//a[contains(@data-test, "company-tile")]/div[2]/div[4]//span[1]')
    company_reviews = company_tile_reviews.text.strip() + " Reviews"

    # Find the company tile salaries element and extract the number of salaries
    company_tile_salaries = driver.find_element(By.XPATH, '//a[contains(@data-test, "company-tile")]/div[2]/div[4]//span[2]')
    company_salaries = company_tile_salaries.text.strip() + " Salaries"

    # Find the company tile jobs element and extract the number of jobs
    company_tile_jobs = driver.find_element(By.XPATH, '//a[contains(@data-test, "company-tile")]/div[2]/div[4]//span[3]')
    company_jobs = company_tile_jobs.text.strip() + " Jobs"

    # Create a dictionary to store the company metadata
    company_metadata = {
        "Company Name": company_name,
        "Industry": company_industry,
        "Headquarters Location": company_headquarters_location,
        "Number of Employees": company_number_of_employees,
        "Number of Reviews": company_reviews,
        "Number of Salaries": company_salaries,
        "Number of Jobs": company_jobs,
        "Glassdoor Company Title": company_glassdoor_title,
        "Glassdoor Overall Rating": company_glassdoor_overall_rating,
        "Glassdoor Company Page URL": company_glassdoor_page_url,
        "Glassdoor Company Logo URL": company_glassdoor_logo_url,
    }

    # Return the company metadata
    return company_metadata

# Example usage
company_metadata = fetch_company_metadata("apple")
print(company_metadata)



In [ ]:

# Define the function to scrape company metadata from a CSV file
def scrape_glassdoor_codes_by_companies_csv(csv_path: str, metadata_path: str = None):
    login_to_glassdoor()  # Log in to Glassdoor

    # Read the CSV file containing company names
    df = pd.read_csv(csv_path, encoding="utf-8")
    company_names = df["Company Name"]

    if metadata_path:
        # If metadata file path is provided, load the existing metadata DataFrame
        metadata_df = pd.read_csv(metadata_path)
    else:
        # Otherwise, create an empty metadata DataFrame
        metadata_df = pd.DataFrame(columns=["Company Name", "Industry", "Headquarters Location", "Number of Employees", "Number of Reviews", "Number of Salaries", "Number of Jobs", "Glassdoor Company Title", "Glassdoor Overall Rating", "Glassdoor Company Page URL", "Glassdoor Company Logo URL"])

    for name in tqdm(company_names):
        try:
            if metadata_path and name in metadata_df["Company Name"].values:
                # Skip if the company metadata already exists in the DataFrame
                continue
            else:
                # Fetch the company metadata and append it to the DataFrame
                metadata_df = metadata_df.append(fetch_company_metadata(name), ignore_index=True)
        except Exception as e:
            # Save the partially scraped metadata in case of an exception
            metadata_df.to_csv(f"data/SP500_glassdoor_metadata_{str(date.today())}.csv", index=False)
            raise e

        time.sleep(1)  # Wait for 1 second between requests

    metadata_df = metadata_df.sort_values("Company Name")  # Sort the metadata DataFrame by company name
    metadata_df.to_csv(f"data/SP500_glassdoor_metadata_{str(date.today())}.csv", index=False)  # Save the metadata to a CSV file

In [ ]:
# The re module in Python provides support for regular expressions, which are used for pattern matching and
# manipulation of strings.
# The function re.search() is a method from the re module that searches for a pattern in a string.
import re

def get_glassdoor_reviews_url_by_company_and_page_number(company_url: str, page_number: int = 1):
    # Extract company name and ID from the provided company URL
    company_name_in_url = re.search('Working-at-(.*)-EI', company_url).group(1)
    company_id_in_url = re.search('-EI_IE(.*)\.11', company_url).group(1)

    if page_number > 1:
        # Construct the reviews URL for a specific company and page number
        glassdoor_reviews_url = f"https://www.glassdoor.com/Reviews/{company_name_in_url}-Reviews-E{company_id_in_url}_P{page_number}.htm?sort.sortType=RD&sort.ascending=true&filter.iso3Language=eng"
    else:
        # Construct the reviews URL for the first page of reviews
        glassdoor_reviews_url = f"https://www.glassdoor.com/Reviews/{company_name_in_url}-Reviews-E{company_id_in_url}.htm?sort.sortType=RD&sort.ascending=true&filter.iso3Language=eng"

    return glassdoor_reviews_url


In [ ]:
def get_number_of_reviews(company_url: str):
    # Get the URL for the company reviews page
    company_reviews_url = get_glassdoor_reviews_url_by_company_and_page_number(company_url)

    repeat = 0
    while repeat < REPEAT_TIMES:
        repeat += 1
        try:
            # Load the company reviews page
            driver.get(company_reviews_url)
            sleep(WAIT_TIME)

            # Find the pagination footer element that contains the number of reviews
            pagination_footer = WebDriverWait(driver, WAIT_TIME).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@data-test, "pagination-footer-text")]')))

            # Extract the number of reviews from the pagination footer text using regular expressions
            number_of_reviews = re.search(' ([^\s]*) Reviews', pagination_footer.text).group(1).replace(",", "").strip()

            # Set repeat to REPEAT_TIMES to break out of the loop
            repeat = REPEAT_TIMES
        except (WebDriverException, StaleElementReferenceException) as e:
            if repeat < REPEAT_TIMES:
                # Retry if there is an exception and we haven't reached the maximum number of retries
                print("Error Message:", e.msg)
                print("Repeat:", repeat)
            else:
                # Raise the exception if the maximum number of retries is reached
                raise e

    # Convert the number of reviews to an integer and return it
    return int(number_of_reviews)


In [ ]:
def get_start_review_number_for_year_in_page(company_url: str, page: int, year: int):
    # Get the URL for the specified company and page number
    company_reviews_url = get_glassdoor_reviews_url_by_company_and_page_number(company_url, page)

    repeat = 0
    while repeat < REPEAT_TIMES:
        repeat += 1
        try:
            # Load the company reviews page
            driver.get(company_reviews_url)
            sleep(WAIT_TIME)

            # Find the element containing the review time spans
            elem = WebDriverWait(driver, WAIT_TIME).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, " fb_reset") and contains(@id, "fb-root")]')))

            # Extract the years from the review time spans
            time_spans = driver.find_elements(By.XPATH, '//ol[contains(@class, "empReviews")]/li//span[contains(@class, "authorJobTitle")]')
            years = [int(re.search(', ([0-9]*) -', span.text).group(1)) for span in time_spans]

            # Set repeat to REPEAT_TIMES to break out of the loop
            repeat = REPEAT_TIMES
        except (WebDriverException, StaleElementReferenceException) as e:
            if repeat < REPEAT_TIMES:
                # Retry if there is an exception and we haven't reached the maximum number of retries
                print("Error Message:", e.msg)
                print("Repeat:", repeat)
            else:
                # Raise the exception if the maximum number of retries is reached
                raise e

    # Find the indices of reviews with years less than, equal to, and greater than the specified year
    years_indices_lt = [i for i, y in enumerate(years) if y < year]
    years_indices_eq = [i for i, y in enumerate(years) if y == year]
    years_indices_gt = [i for i, y in enumerate(years) if y > year]

    if len(years_indices_eq) > 0:
        # If there are reviews with the specified year, return the index of the next review
        return years_indices_eq[0] + 1
    elif len(years_indices_lt) > 0 and len(years_indices_gt) > 0:
        # If there are reviews before and after the specified year, return -1
        return -1
    elif len(years_indices_gt) > 0:
        # If there are only reviews after the specified year, return 0
        return 0
    else:  # len(years_indices_lt) > 0:
        # If there are only reviews before the specified year, return 11
        return 11


In [ ]:
def get_end_review_number_for_year_in_page(company_url: str, page: int, year: int):
    # Get the URL for the specified company and page number
    company_reviews_url = get_glassdoor_reviews_url_by_company_and_page_number(company_url, page)

    repeat = 0
    while repeat < REPEAT_TIMES:
        repeat += 1
        try:
            # Load the company reviews page
            driver.get(company_reviews_url)
            sleep(WAIT_TIME)

            # Find the element containing the review time spans
            elem = WebDriverWait(driver, WAIT_TIME).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, " fb_reset") and contains(@id, "fb-root")]')))

            # Extract the years from the review time spans
            time_spans = driver.find_elements(By.XPATH, '//ol[contains(@class, "empReviews")]/li//span[contains(@class, "authorJobTitle")]')
            years = [int(re.search(', ([0-9]*) -', span.text).group(1)) for span in time_spans]

            # Set repeat to REPEAT_TIMES to break out of the loop
            repeat = REPEAT_TIMES
        except (WebDriverException, StaleElementReferenceException) as e:
            if repeat < REPEAT_TIMES:
                # Retry if there is an exception and we haven't reached the maximum number of retries
                print("Error Message:", e.msg)
                print("Repeat:", repeat)
            else:
                # Raise the exception if the maximum number of retries is reached
                raise e

    # Find the indices of reviews with years less than, equal to, and greater than the specified year
    years_indices_lt = [i for i, y in enumerate(years) if y < year]
    years_indices_eq = [i for i, y in enumerate(years) if y == year]
    years_indices_gt = [i for i, y in enumerate(years) if y > year]

    if len(years_indices_eq) > 0:
        # If there are reviews with the specified year, return the index of the last review
        return years_indices_eq[-1] + 1
    elif len(years_indices_lt) > 0 and len(years_indices_gt) > 0:
        # If there are reviews before and after the specified year, return -1
        return -1
    elif len(years_indices_gt) > 0:
        # If there are only reviews after the specified year, return 0
        return 0
    else:  # len(years_indices_lt) > 0:
        # If there are only reviews before the specified year, return 11
        return 11


In [ ]:
def get_start_review_page_number_for_year_by_company_url(company_url: str, year: int, sp: int = None, ep: int = None):
    # Login to Glassdoor
    login_to_glassdoor()

    # Get the total number of reviews for the company
    number_of_reviews = get_number_of_reviews(company_url)

    # Calculate the total number of pages
    number_of_pages = number_of_reviews // 10

    # Set the start page and end page if not provided
    if not sp:
        sp = 1
    if not ep:
        ep = number_of_pages

    # Initialize variables for left page, middle page, and right page
    lp, mp, rp = sp, ((ep - sp) // 2) + sp, ep

    # Initialize variables for left index, middle index, and right index
    li, mi, ri = -2, -2, -2

    # Initialize a dictionary to store the start page and start index
    start_dict = {}

    # Perform binary search to find the start page and start index
    while lp <= rp:
        # Check the first page
        li = get_start_review_number_for_year_in_page(company_url, lp, year)
        if li != 11:
            start_dict["start_page"], start_dict["start_index"] = lp, li
            break

        # Check the last page
        ri = get_start_review_number_for_year_in_page(company_url, rp, year)
        if ri != 0 and ri != 1:
            start_dict["start_page"], start_dict["start_index"] = rp, ri
            break

        # Check the middle page
        mi = get_start_review_number_for_year_in_page(company_url, mp, year)
        if (2 <= mi and mi <= 10) or mi == -1:
            start_dict["start_page"], start_dict["start_index"] = mp, mi
            break
        elif mi == 0:
            lp = lp + 1
            rp = mp - 1
        elif mi == 1:
            lp = lp + 1
            rp = mp
        else:  # mi == 11
            lp = mp + 1
            if ri == 0:
                rp = rp - 1
        mp = (lp + rp) // 2

    return start_dict


In [ ]:
def get_end_review_page_number_for_year_by_company_url(company_url: str, year: int, sp: int = None, ep: int = None):
    # Login to Glassdoor
    login_to_glassdoor()

    # Get the total number of reviews for the company
    number_of_reviews = get_number_of_reviews(company_url)

    # Calculate the total number of pages
    number_of_pages = number_of_reviews // 10

    # Set the start page and end page if not provided
    if not sp:
        sp = 1
    if not ep:
        ep = number_of_pages

    # Initialize variables for left page, middle page, and right page
    lp, mp, rp = sp, ((ep - sp) // 2) + sp, ep

    # Initialize variables for left index, middle index, and right index
    li, mi, ri = -2, -2, -2

    # Initialize a dictionary to store the end page and end index
    end_dict = {}

    # Perform binary search to find the end page and end index
    while lp <= rp:
        # Check the first page
        li = get_end_review_number_for_year_in_page(company_url, lp, year)
        if li != 10 and li != 11:
            end_dict["end_page"], end_dict["end_index"] = lp, li
            break

        # Check the last page
        ri = get_end_review_number_for_year_in_page(company_url, rp, year)
        if ri != 0:
            end_dict["end_page"], end_dict["end_index"] = rp, ri
            break

        # Check the middle page
        mi = get_end_review_number_for_year_in_page(company_url, mp, year)
        if (1 <= mi and mi <= 9) or mi == -1:
            end_dict["end_page"], end_dict["end_index"] = mp, mi
            break
        elif mi == 10:
            lp = mp
            rp = rp - 1
        elif mi == 11:
            lp = mp + 1
            rp = rp - 1
        else:  # mi == 0
            if li == 11:
                lp = lp + 1
            rp = mp - 1
        mp = (lp + rp) // 2

    return end_dict


In [ ]:
def scrap_glassdoor_start_and_end_review_numbers_by_range(csv_path: str, start_year: int, end_year: int, metadata_path: str = None):
  # Login to Glassdoor
  login_to_glassdoor()

  # Read the CSV file
  df = pd.read_csv(csv_path, encoding="utf-8")

  # Read the metadata if provided, otherwise create a new DataFrame
  if metadata_path:
    metadata_df = pd.read_csv(metadata_path)
  else:
    metadata_df = pd.DataFrame(columns=["Company Name", "Total Number of Reviews", "Start Year", "Page of First Review", "Index of First Review", "End Year", "Page of Last Review", "Index of Last Review", "Number of Reviews in Time Frame"])

  # Get company names and Glassdoor URLs
  company_names = df["Company Name"]
  glassdoor_urls = df["Glassdoor Company Page URL"]

  # Iterate over each company
  for url, name in zip(tqdm(glassdoor_urls), company_names):
    try:
      if metadata_path and name in metadata_df["Company Name"].values:
        pass
      else:
        # Get the total number of reviews for the company
        total_number_of_reviews = get_number_of_reviews(url)

        # Get the start and end review page numbers for the specified years
        start_dict = get_start_review_page_number_for_year_by_company_url(url, start_year)
        print(start_dict)
        end_dict = get_end_review_page_number_for_year_by_company_url(url, end_year)
        print(end_dict)

        # Add the metadata to the DataFrame
        metadata_df.loc[len(metadata_df)] = {
          "Company Name": name,
          "Total Number of Reviews": total_number_of_reviews,
          "Start Year": start_year,
          "Page of First Review": start_dict["start_page"],
          "Index of First Review": start_dict["start_index"],
          "End Year": end_year,
          "Page of Last Review": end_dict["end_page"],
          "Index of Last Review": end_dict["end_index"],
          "Number of Reviews in Time Frame": end_dict["end_page"]*10 + end_dict["end_index"] - start_dict["start_page"]*10 - start_dict["start_index"] + 1
        }

        # Save the metadata to a CSV file
        metadata_df.to_csv(f"data/SP500_glassdoor_metadata_{str(date.today())}.csv", index=False)

    except KeyboardInterrupt as e:
      metadata_df.to_csv(f"data/SP500_glassdoor_metadata_{str(date.today())}.csv", index=False)
      raise e

    except Exception as e:
      metadata_df.to_csv(f"data/SP500_glassdoor_metadata_{str(date.today())}.csv", index=False)
      raise e

  # Sort the metadata DataFrame by company name
  metadata_df = metadata_df.sort_values("Company Name")

  # Save the final metadata to a CSV file
  metadata_df.to_csv(f"data/SP500_glassdoor_metadata_{str(date.today())}.csv", index=False)


In [ ]:
def check_reviews_metadata(csv_path: str, metadata_path: str):
    login_to_glassdoor()  # Login to Glassdoor using the `login_to_glassdoor()` function

    df = pd.read_csv(csv_path, encoding="utf-8")  # Read the input CSV file containing company information
    df['start_ok'] = None  # Create new columns to store the "start_ok" and "end_ok" values
    df['end_ok'] = None

    metadata_df = pd.read_csv(metadata_path, encoding="utf-8")  # Read the metadata CSV file

    company_names = df["Company Name"]  # Extract the company names from the DataFrame
    glassdoor_urls = df["Glassdoor Company Page URL"]  # Extract the Glassdoor company page URLs from the DataFrame

    for i, (url, name) in enumerate(zip(tqdm(glassdoor_urls), company_names)):
        repeat = 0
        while repeat < REPEAT_TIMES:
            repeat += 1
            try:
                # Extract the start and end page numbers and index values from the metadata DataFrame
                start_page = metadata_df.loc[i, "Page of First Review"]
                start_index = metadata_df.loc[i, "Index of First Review"]
                end_page = metadata_df.loc[i, "Page of Last Review"]
                end_index = metadata_df.loc[i, "Index of Last Review"]

                # Generate the URLs for the start and end reviews pages using the company URL and page numbers
                start_reviews_page_url = get_glassdoor_reviews_url_by_company_and_page_number(url, start_page)
                before_start_reviews_page_url = get_glassdoor_reviews_url_by_company_and_page_number(url, start_page - 1)
                end_reviews_page_url = get_glassdoor_reviews_url_by_company_and_page_number(url, end_page)
                after_end_reviews_page_url = get_glassdoor_reviews_url_by_company_and_page_number(url, end_page + 1)

                # Check the "start_ok" condition based on the start index and years of the reviews
                if start_index <= 0 or start_index >= 11:
                    df.loc[i, 'start_ok'] = False
                elif start_index > 1:
                    # Navigate to the start reviews page and extract the year of the start review and the preceding review
                    driver.get(start_reviews_page_url)
                    sleep(WAIT_TIME)
                    date_and_job_span = driver.find_element(By.XPATH,
                                                            f'//ol[contains(@class, "empReviews")]/li[{start_index}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                    start_year = date_and_job_span.text.strip().split("-")[0].strip().split(",")[1].strip()
                    date_and_job_span = driver.find_element(By.XPATH,
                                                            f'//ol[contains(@class, "empReviews")]/li[{start_index - 1}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                    before_start_year = date_and_job_span.text.strip().split("-")[0].strip().split(",")[1].strip()

                    # Check if the start review is from the year 2017 and the preceding review is not from 2017
                    if start_year == '2017' and before_start_year != '2017':
                        df.loc[i, 'start_ok'] = True
                    else:
                        df.loc[i, 'start_ok'] = False
                elif start_index == 1:
                    # Navigate to the start and preceding reviews pages and extract the years of the reviews
                    driver.get(start_reviews_page_url)
                    sleep(WAIT_TIME)
                    date_and_job_span = driver.find_element(By.XPATH,
                                                            f'//ol[contains(@class, "empReviews")]/li[{start_index}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                    start_year = date_and_job_span.text.strip().split("-")[0].strip().split(",")[1].strip()
                    driver.get(before_start_reviews_page_url)
                    sleep(WAIT_TIME)
                    date_and_job_span = driver.find_element(By.XPATH,
                                                            f'//ol[contains(@class, "empReviews")]/li[10]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                    before_start_year = date_and_job_span.text.strip().split("-")[0].strip().split(",")[1].strip()

                    # Check if the start review is from the year 2017 and the preceding review is not from 2017
                    if start_year == '2017' and before_start_year != '2017':
                        df.loc[i, 'start_ok'] = True
                    else:
                        df.loc[i, 'start_ok'] = False
                else:
                    df.loc[i, 'start_ok'] = False

                # Check the "end_ok" condition based on the end index and years of the reviews
                if end_index <= 0 or end_index >= 11:
                    df.loc[i, 'end_ok'] = False
                elif end_index < 10:
                    # Navigate to the end reviews page and extract the year of the end review and the following review
                    driver.get(end_reviews_page_url)
                    sleep(WAIT_TIME)
                    date_and_job_span = driver.find_element(By.XPATH,
                                                            f'//ol[contains(@class, "empReviews")]/li[{end_index}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                    end_year = date_and_job_span.text.strip().split("-")[0].strip().split(",")[1].strip()
                    date_and_job_span = driver.find_element(By.XPATH,
                                                            f'//ol[contains(@class, "empReviews")]/li[{end_index + 1}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                    after_end_year = date_and_job_span.text.strip().split("-")[0].strip().split(",")[1].strip()

                    # Check if the end review is from the year 2019 and the following review is not from 2019
                    if end_year == '2019' and after_end_year != '2019':
                        df.loc[i, 'end_ok'] = True
                    else:
                        df.loc[i, 'end_ok'] = False
                elif end_index == 10:
                    # Navigate to the end and following reviews pages and extract the years of the reviews
                    driver.get(end_reviews_page_url)
                    sleep(WAIT_TIME)
                    date_and_job_span = driver.find_element(By.XPATH,
                                                            f'//ol[contains(@class, "empReviews")]/li[{end_index}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                    end_year = date_and_job_span.text.strip().split("-")[0].strip().split(",")[1].strip()
                    driver.get(after_end_reviews_page_url)
                    sleep(WAIT_TIME)
                    date_and_job_span = driver.find_element(By.XPATH,
                                                            f'//ol[contains(@class, "empReviews")]/li[1]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                    after_end_year = date_and_job_span.text.strip().split("-")[0].strip().split(",")[1].strip()

                    # Check if the end review is from the year 2019 and the following review is not from 2019
                    if end_year == '2019' and after_end_year != '2019':
                        df.loc[i, 'end_ok'] = True
                    else:
                        df.loc[i, 'end_ok'] = False
                else:
                    df.loc[i, 'end_ok'] = False

            except (WebDriverException, StaleElementReferenceException) as e:
                if repeat < REPEAT_TIMES:
                    print("Error Message:", e.msg)
                    print("Repeat:", repeat)
                else:
                    raise e

        df.to_csv("ok.csv")  # Save the updated DataFrame to a new CSV file


In [ ]:
#summary:
#  The function `scrape_glassdoor_reviews_in_page` scrapes reviews from a specific page on Glassdoor for a given company. Here's a summary of the function:
# - It takes four parameters: `page_url` (the URL of the page to scrape), `company_name` (the name of the company), `start_index` (the index of the first review to scrape on the page), and `end_index` (the index of the last review to scrape on the page).
# - The function initializes a repeat counter and creates an empty DataFrame called `reviews_df` to store the scraped review data.
# - It enters a loop that will execute a maximum of `REPEAT_TIMES` times (a constant value).
# - Inside the loop, it attempts to scrape the page and retrieve the desired review information.
# - If successful, the function iterates over the specified range of review indices and extracts various attributes for each review, such as scores, titles, dates, job descriptions, pros, cons, and helpful rates. The extracted information is added to the `reviews_df` DataFrame.
# - If any exception occurs during the scraping process (e.g., WebDriverException, StaleElementReferenceException, NoSuchElementException, TimeoutException), the function handles it by either retrying (if the repeat counter is less than `REPEAT_TIMES`) or raising the exception.
# - Once the loop completes or an exception is raised, the function returns the `reviews_df` DataFrame containing the scraped review data.
# This function is likely part of a larger web scraping process for collecting reviews from multiple pages on Glassdoor for different companies.
def scrape_glassdoor_reviews_in_page(page_url: str, company_name: str, start_index: int = 1, end_index: int = 10):
    repeat = 0

    while repeat < REPEAT_TIMES:
        reviews_df = pd.DataFrame(columns=["Company Name", "Total Score", "Work/Life Balance Score", "Culture & Values Score", "Diversity & Inclusion Score", "Career Opportunities Score", "Compensation and Benefits Score", "Senior Management Score", "Current/Former Employee", "Employment Length", "Review Title", "Review Date", "Job Description", "Recommend Level", "CEO Approval Level", "Business Outlook Level", "Pros", "Cons", "Helpful Rate"])
        repeat += 1
        try:
            driver.get(page_url)
            sleep(WAIT_TIME)
            elem = WebDriverWait(driver, WAIT_TIME).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, " fb_reset") and contains(@id, "fb-root")]')))
            for i in range(start_index, end_index+1):
                review_dict = {}
                review_dict["Company Name"] = company_name
                total_score_div = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[1]/div/div/div/div')
                review_dict["Total Score"] = STARS_CLASSES_DICT[total_score_div.get_attribute('class').split(" ")[0]]
                work_life_balance_score_divs = driver.find_elements(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[1]/div/div/aside/div/div/ul/li/div[contains(text(), "Work/Life Balance")]/following-sibling::div')
                review_dict["Work/Life Balance Score"] = STARS_CLASSES_DICT[work_life_balance_score_divs[0].get_attribute('class').split(" ")[0]] if len(work_life_balance_score_divs) == 1 else None
                culture_and_values_score_divs = driver.find_elements(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[1]/div/div/aside/div/div/ul/li/div[contains(text(), "Culture & Values")]/following-sibling::div')
                review_dict["Culture & Values Score"] = STARS_CLASSES_DICT[work_life_balance_score_divs[0].get_attribute('class').split(" ")[0]] if len(work_life_balance_score_divs) == 1 else None
                diversity_and_inclusion_score_divs = driver.find_elements(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[1]/div/div/aside/div/div/ul/li/div[contains(text(), "Diversity & Inclusion")]/following-sibling::div')
                review_dict["Diversity & Inclusion Score"] = STARS_CLASSES_DICT[diversity_and_inclusion_score_divs[0].get_attribute('class').split(" ")[0]] if len(diversity_and_inclusion_score_divs) == 1 else None
                career_opportunities_score_divs = driver.find_elements(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[1]/div/div/aside/div/div/ul/li/div[contains(text(), "Career Opportunities")]/following-sibling::div')
                review_dict["Career Opportunities Score"] = STARS_CLASSES_DICT[career_opportunities_score_divs[0].get_attribute('class').split(" ")[0]] if len(career_opportunities_score_divs) == 1 else None
                compensation_and_benefits_score_divs = driver.find_elements(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[1]/div/div/aside/div/div/ul/li/div[contains(text(), "Compensation and Benefits")]/following-sibling::div')
                review_dict["Compensation and Benefits Score"] = STARS_CLASSES_DICT[compensation_and_benefits_score_divs[0].get_attribute('class').split(" ")[0]] if len(compensation_and_benefits_score_divs) == 1 else None
                senior_management_score_divs = driver.find_elements(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[1]/div/div/aside/div/div/ul/li/div[contains(text(), "Senior Management")]/following-sibling::div')
                review_dict["Senior Management Score"] = STARS_CLASSES_DICT[senior_management_score_divs[0].get_attribute('class').split(" ")[0]] if len(senior_management_score_divs) == 1 else None
                employment_span = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[1]/div/span')
                review_dict["Current/Former Employee"] = employment_span.text.strip().split(",")[0].strip()
                review_dict["Employment Length"] = employment_span.text.strip().split(",")[1].strip() if len(employment_span.text.strip().split(",")) == 2 else None
                title_h2 = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/h2')
                review_dict["Review Title"] = title_h2.text.strip()
                date_and_job_span = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/span')
                review_dict["Review Date"] = date_and_job_span.text.strip().split("-")[0].strip()
                review_dict["Job Description"] = date_and_job_span.text.strip().split("-")[1].strip() if len(date_and_job_span.text.strip().split("-")) == 2 and len(date_and_job_span.text.strip().split("-")[1].strip()) > 0 else None
                recommend_span = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/div/div/div/span[contains(text(), "Recommend")]/preceding-sibling::span')
                review_dict["Recommend Level"] = V_X_DICT[recommend_span.get_attribute('class').split(" ")[1]]
                ceo_approval_span = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[2]/div/div[1]/div/div/div/span[contains(text(), "CEO Approval")]/preceding-sibling::span')
                review_dict["CEO Approval Level"] = V_X_DICT[ceo_approval_span.get_attribute('class').split(" ")[1]]
                buisness_outlook_span = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[2]/div[1]/div/div/div/div/span[contains(text(), "Business Outlook")]/preceding-sibling::span')
                review_dict["Business Outlook Level"] = V_X_DICT[buisness_outlook_span.get_attribute('class').split(" ")[1]]
                pros_span = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[2]/div/div[2]//span[contains(@data-test, "pros")]')
                review_dict["Pros"] = pros_span.text.strip()
                cons_span = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[2]/div/div[2]//span[contains(@data-test, "cons")]')
                review_dict["Cons"] = cons_span.text.strip()
                helpful_rate_div = driver.find_element(By.XPATH, f'//ol[contains(@class, "empReviews")]/li[{i}]//div[contains(@class, "gdReview")]/div[2]/div/div[2]/div[contains(@class, "common__EiReviewDetailsStyle__socialHelpfulcontainer pt-std")]')
                review_dict["Helpful Rate"] = int(helpful_rate_div.text.strip().split(" ")[0]) if 'Be' not in helpful_rate_div.text.strip().split(" ")[0] else 0
                reviews_df.loc[len(reviews_df)] = review_dict

            repeat = REPEAT_TIMES
        except (WebDriverException, StaleElementReferenceException, NoSuchElementException, TimeoutException) as e:
            if repeat < REPEAT_TIMES:
                print("Error Message:", e.msg)
                print("Repeat:", repeat)
            else:
                raise e

    return reviews_df

In [ ]:
def scrape_glassdoor_reviews(csv_path: str, metadata_path: str):
    # Log in to Glassdoor
    login_to_glassdoor()

    # Read the CSV file containing company data
    df = pd.read_csv(csv_path)

    # Read the metadata CSV file
    metadata_df = pd.read_csv(metadata_path)

    # Get the list of company names
    company_names = metadata_df["Company Name"]

    # Filter the Glassdoor URLs based on the company names
    glassdoor_urls = df[df["Company Name"].isin(company_names)]["Glassdoor Company Page URL"]

    # Iterate over the URLs and company names
    for url, name in zip(tqdm(glassdoor_urls), company_names):
        temp_reviews_dfs = []

        # Define the path for the review CSV file
        review_csv_path = f"data/reviews/{name}.csv"

        # Check if the review CSV file already exists
        if os.path.exists(review_csv_path):
            # If it exists, load the existing reviews DataFrame
            reviews_df = pd.read_csv(review_csv_path)
        else:
            # If it doesn't exist, create an empty reviews DataFrame with column names
            reviews_df = pd.DataFrame(columns=["Company Name", "Total Score", "Work/Life Balance Score", "Culture & Values Score", "Diversity & Inclusion Score", "Career Opportunities Score", "Compensation and Benefits Score", "Senior Management Score", "Current/Former Employee", "Employment Length", "Review Title", "Review Date", "Job Description", "Recommend Level", "CEO Approval Level", "Business Outlook Level", "Pros", "Cons", "Helpful Rate"])

        # Determine the starting and ending page numbers and indices for reviews
        start_page = metadata_df[metadata_df["Company Name"] == name].reset_index(drop=True).loc[0, "Page of First Review"] + len(reviews_df) // 10
        start_index = metadata_df[metadata_df["Company Name"] == name].reset_index(drop=True).loc[0, "Index of First Review"] if len(reviews_df) == 0 else len(reviews_df) % 10
        end_page = metadata_df[metadata_df["Company Name"] == name].reset_index(drop=True).loc[0, "Page of Last Review"]
        end_index = metadata_df[metadata_df["Company Name"] == name].reset_index(drop=True).loc[0, "Index of Last Review"]

        # Iterate over the pages and scrape reviews for each page
        for i, page in enumerate(range(start_page, end_page+1)):
            # Get the URL for the reviews page of the current company and page number
            reviews_page_url = get_glassdoor_reviews_url_by_company_and_page_number(url, page)

            # Set the start and end indices for reviews on the current page
            si = 1
            ei = 10
            if page == start_page:
                si = start_index
            if page == end_page:
                ei = end_index

            # Scrape the reviews on the current page and append them to the temporary list
            temp_reviews_dfs.append(scrape_glassdoor_reviews_in_page(reviews_page_url, name, start_index=si, end_index=ei))

            # Check if the temporary list has reached a batch size (every 10 pages)
            if i % 10 == 0:
                # Concatenate the temporary list of reviews DataFrames and save them to the review CSV file
                reviews_df = pd.concat([reviews_df] + temp_reviews_dfs)
                reviews_df.to_csv(review_csv_path, index=False)
                temp_reviews_dfs = []

        # Concatenate the remaining reviews DataFrames in the temporary list and save them to the review CSV file
        reviews_df = pd.concat([reviews_df] + temp_reviews_dfs)
        reviews_df.to_csv(review_csv_path, index=False)


In [ ]:
def clean_reviews(old_dir: str, new_dir: str, years: list = ['2017', '2018', '2019']):
    # Create an empty DataFrame to store metadata
    df = pd.DataFrame(columns=["Company Name", "Number of Reviews"])

    # Get a list of CSV files in the old directory
    files = glob(os.path.join(old_dir, "*.csv"))

    # Iterate over the files
    for f in tqdm(files):
        # Extract the company name from the file path
        company_name = '.'.join(f.split('\\')[-1].split('.')[0:-1]).strip()

        # Read the CSV file into a DataFrame
        company_df = pd.read_csv(f)

        # Filter the DataFrame to include only reviews from the specified years
        bool_series = company_df['Review Date'].apply(lambda x: x.split(',')[-1].strip()).isin(years)
        company_df = company_df.loc[bool_series].reset_index(drop=True)

        # Save the filtered DataFrame to a new CSV file in the new directory
        company_df.to_csv(os.path.join(new_dir, f"{company_name}.csv"), index=False)

        # Update the metadata DataFrame with the company name and the number of reviews
        df.loc[len(df)] = {'Company Name': company_name, 'Number of Reviews': len(company_df)}

    # Save the metadata DataFrame to a CSV file
    df.to_csv('SP500_2020_reviews_metadata.csv', index=False)


## Run

In [ ]:
CSV_PATH = "data/SP500_2020_components.csv"
META_DATA_PATH = "data/SP500_glassdoor_metadata_2022-06-27.csv"
# scrape_glassdoor_codes_by_companies_csv(CSV_PATH)

In [ ]:
CSV_PATH = "data/SP500_2020_glassdoor_metadata.csv"
META_DATA_PATH = "data/SP500_glassdoor_metadata_2022-06-27.csv"
# scrap_glassdoor_start_and_end_review_numbers_by_range(CSV_PATH, 2017, 2019, META_DATA_PATH)

In [ ]:
CSV_PATH = "SP500_2020_glassdoor_metadata_part.csv"
META_DATA_PATH = "SP500_2020_reviews_glassdoor_metadata_part.csv"
# check_reviews_metadata(CSV_PATH, META_DATA_PATH)

In [ ]:
CSV_PATH = "data/SP500_2020_glassdoor_metadata.csv"
META_DATA_PATH = "data/SP500_2020_reviews_glassdoor_metadata.csv"
# scrape_glassdoor_reviews(CSV_PATH, META_DATA_PATH)

In [ ]:
OLD_DIR = "data/reviews_raw"
NEW_DIR = "data/reviews_clean"
# clean_reviews(OLD_DIR, NEW_DIR)

# Financialmodelingprep API

## Imports

In [ ]:
import requests
import pandas as pd
from tqdm.auto import tqdm

## Utils

In [ ]:
def get_balance_sheet_stmts(stock, period="Q4", save_dir=None, name=None):
    # Send a GET request to the API to fetch the balance sheet statement for the specified stock
    bs = requests.get(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{stock}?period=quarter&limit=30&apikey=0239314287816adcc243809e01bf1192")

    # Convert the response to JSON format
    bs = bs.json()

    # Print the stock symbol
    print(stock)

    # Create a DataFrame from the JSON data
    df = pd.DataFrame(bs)

    # Check if the DataFrame contains the 'date' column
    if "date" not in df.columns:
        # If the 'date' column is missing, print the JSON data and display the DataFrame
        print(bs)
        display(df)
    else:
        # If the 'date' column exists
        if period:
            # Filter the DataFrame based on the specified period
            df = df[df["period"] == period]

        if save_dir:
            # If a save directory is provided
            if not name:
                # If no name is specified, use the stock symbol as the file name
                name = stock

            # Save the DataFrame as a CSV file in the specified directory
            df.to_csv(f"{save_dir}/{name}.csv", index=False)

    # Return the DataFrame
    return df


In [ ]:
def get_market_cap(stock, month_day=["12-31", "12-30", "12-29", "12-28", "12-27", "12-26", "12-25", "12-24", "12-23", "12-22", "12-21", "12-20"], save_dir=None, name=None):
    # Send a GET request to the API to fetch the historical market capitalization data for the specified stock
    p = requests.get(f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{stock}?period=quarter&limit=2500&apikey=0239314287816adcc243809e01bf1192")

    # Convert the response to JSON format
    p = p.json()

    # Create a DataFrame from the JSON data
    df = pd.DataFrame(p)

    # Check if the DataFrame contains the 'date' column
    if "date" not in df.columns:
        # If the 'date' column is missing, print the JSON data and display the DataFrame
        print(p)
        display(df)
    else:
        # If the 'date' column exists
        if month_day:
            # Create a new DataFrame to store filtered data
            new_df = pd.DataFrame(columns=df.columns)

            # Extract month and day from the 'date' column and create 'month_day' and 'year' columns
            df["month_day"] = df["date"].apply(lambda x: "-".join(x.split("-")[1:]))
            df["year"] = df["date"].apply(lambda x: x.split("-")[0])

            # Iterate over unique years in the DataFrame
            for year in df["year"].unique():
                # Filter the DataFrame for a specific year
                partial_df = df[df["year"] == year]

                # Iterate over the specified month_day values
                for md in month_day:
                    # Check if the month_day value exists in the filtered DataFrame
                    if md in partial_df['month_day'].unique():
                        # Append the first row with the matching month_day value to the new DataFrame
                        new_df.loc[len(new_df)] = partial_df[partial_df['month_day'] == md].reset_index(drop=True).iloc[0]
                        break

            # Update the DataFrame with the filtered data
            df = new_df

        if save_dir:
            # If a save directory is provided
            if not name:
                # If no name is specified, use the stock symbol as the file name
                name = stock

            # Save the DataFrame as a CSV file in the specified directory
            df.to_csv(f"{save_dir}/{name}.csv", index=False)

    # Return the DataFrame
    return df


In [ ]:
import pandas as pd
from tqdm import tqdm

def save_balance_stmt_and_market_cap(meta_file):
    # Read the metadata file into a DataFrame
    df = pd.read_csv(meta_file)

    # Iterate over each row in the DataFrame
    for i in tqdm(range(len(df))):
        symbol = df.loc[i, "Symbol"]
        company_name = df.loc[i, "Company Name"]

        # Print the progress
        if i > 320:
            print(i, company_name, symbol)

        # Get the balance sheet statements for the company
        bs_df = get_balance_sheet_stmts(symbol,
                                        period="Q4",
                                        save_dir="/content/drive/MyDrive/glassdoor_research/Scraping/data/financial_data/balance_sheet_stmts",
                                        name=company_name)

        # If the balance sheet DataFrame is empty, continue to the next iteration
        if len(bs_df) == 0:
            continue

        # Get the market capitalization data for the company
        get_market_cap(symbol,
                       month_day=["12-31", "12-30", "12-29", "12-28", "12-27", "12-26", "12-25", "12-24", "12-23", "12-22", "12-21", "12-20"],
                       save_dir="/content/drive/MyDrive/glassdoor_research/Scraping/data/financial_data/market_cap",
                       name=company_name)


In [ ]:
def get_tobins_q_ver3(meta_file):
    # Read the metadata file into a DataFrame
    meta_df = pd.read_csv(meta_file)

    # Create an empty DataFrame to store the results
    res_df = pd.DataFrame(columns=["Symbol", "Company Name", "Date", "Tobin's Q"])

    # Iterate over each row in the metadata DataFrame
    for i in tqdm(range(len(meta_df))):
        symbol = meta_df.loc[i, "Symbol"]
        company_name = meta_df.loc[i, "Company Name"]

        # Define the paths for the balance sheet and market cap data
        bs_path = f"/content/drive/MyDrive/glassdoor_research/Scraping/data/financial_data/balance_sheet_stmts/{company_name}.csv"
        mc_path = f"/content/drive/MyDrive/glassdoor_research/Scraping/data/financial_data/market_cap/{company_name}.csv"

        # Check if the balance sheet file exists
        if os.path.isfile(bs_path):
            bs_df = pd.read_csv(bs_path).reset_index(drop=True)
        else:
            continue

        # Check if the market cap file exists
        if os.path.isfile(mc_path):
            mc_df = pd.read_csv(mc_path).reset_index(drop=True)
        else:
            continue

        # Extract the year from the date column in market cap DataFrame
        mc_df["year"] = mc_df["date"].apply(lambda x: x.split("-")[0])

        # Extract the year from the date column in balance sheet DataFrame
        bs_df['year'] = bs_df["date"].apply(lambda x: x.split("-")[0])

        # Iterate over each date and year in the market cap DataFrame
        for date, year in zip(mc_df["date"], mc_df["year"]):
            # Filter the balance sheet DataFrame for the corresponding year
            bs_df_partial = bs_df[bs_df["year"] == year].reset_index(drop=True)

            # Continue to the next iteration if the balance sheet DataFrame is empty
            if len(bs_df_partial) == 0:
                continue

            # Retrieve the total stockholders' equity from the balance sheet DataFrame
            total_stockholders_equity = bs_df_partial.loc[0, "totalLiabilitiesAndStockholdersEquity"]

            # Filter the market cap DataFrame for the corresponding year
            mc_df_partial = mc_df[mc_df["year"] == year].reset_index(drop=True)

            # Retrieve the market cap from the market cap DataFrame
            market_cap = mc_df_partial.loc[0, "marketCap"]

            # Calculate Tobin's Q
            tobins_q = market_cap / total_stockholders_equity

            # Check if Tobin's Q is negative
            if tobins_q < 0:
                print(market_cap, total_stockholders_equity)
                display(bs_df)
                raise ValueError("stop")

            # Append the result to the result DataFrame
            res_df.loc[len(res_df)] = [symbol, company_name, date, tobins_q]

    # Save the result DataFrame to a CSV file
    res_df.to_csv(f"/content/drive/MyDrive/glassdoor_research/Scraping/data/SP500_2020_financialmodelingprep_data_ver3.csv", index=False)


## Run

In [ ]:
META_FILE = "/content/drive/MyDrive/glassdoor_research/Scraping/data/SP500_2020_wikipedia_metadata.csv"
# save_balance_stmt_and_market_cap(META_FILE)

In [ ]:
META_FILE = "/content/drive/MyDrive/glassdoor_research/Scraping/data/SP500_2020_wikipedia_metadata.csv"
# get_tobins_q_ver3(META_FILE)

# ML and DS

## Imports

In [ ]:
import os
import random
import statistics
import numpy as np
import pandas as pd
from glob import glob
from tqdm.auto import tqdm
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

## Constants

In [ ]:
X_NUMERIC_COLUMNS = ["Total Score", "Work/Life Balance Score", "Culture & Values Score", "Career Opportunities Score", "Compensation and Benefits Score", "Senior Management Score", "Helpful Rate"]
X_CATEGORICAL_COLUMNS = ["Current/Former Employee", "Employment Length", "Recommend Level", "CEO Approval Level", "Business Outlook Level"]
X_TEXT_COLUMNS = ["Review Title", "Pros", "Cons"]
Y_COLUMN = "Tobin's Q"
DROP_COLUMNS = ["Diversity & Inclusion Score"]

## Data

In [ ]:

def get_company_data(csv_dir, take_n_random_comments=None, drop_columns=None, na_action=None):
    # Get a list of file paths in the CSV directory
    file_paths = glob(os.path.join(csv_dir, "*.csv"))

    # Initialize an empty list to store the DataFrames
    df_list = []

    # Iterate over each file path
    for file_path in tqdm(file_paths):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Optionally, select a random subset of comments from each DataFrame
        if take_n_random_comments:
            if len(df) < take_n_random_comments:
                continue
            else:
                df = df.sample(n=take_n_random_comments)

        # Append the DataFrame to the list
        df_list.append(df)

    # Concatenate all the DataFrames into a single DataFrame
    results_df = pd.concat(df_list).reset_index(drop=True)

    # Drop specified columns from the resulting DataFrame
    if drop_columns:
        results_df = results_df.drop(drop_columns, axis=1)

    # Handle missing values based on the specified action
    if na_action == "drop":
        results_df = results_df.dropna(axis=0)
    elif na_action == "replace":
        # Iterate over each column in the DataFrame
        for j in tqdm(range(len(results_df.columns))):
            # Iterate over each row in the DataFrame
            for i in range(len(results_df)):
                # Check if the value is NaN
                if pd.isna(results_df.iloc[i, j]):
                    # Get the company name for reference
                    company_name = results_df.loc[i, "Company Name"]
                    try:
                        # Get the non-null values for the same company and column
                        vals = sorted(results_df[results_df["Company Name"] == company_name].iloc[:, j].dropna())
                        # Randomly select a value from the non-null values
                        val = random.sample(vals, k=1)[0]
                    except:
                        print(company_name, i, j)
                        print(results_df[results_df["Company Name"] == company_name].iloc[:, j])
                    # Replace the NaN value with the selected value
                    results_df.iloc[i, j] = val
    else:
        pass

    # Return the resulting DataFrame
    return results_df


In [ ]:
REVIEWS_DATA_DIR = "/content/drive/MyDrive/glassdoor_research/Scraping/data/reviews_clean"
results_df = get_company_data(REVIEWS_DATA_DIR, drop_columns=DROP_COLUMNS)
results_df

  0%|          | 0/492 [00:00<?, ?it/s]

,Company Name,Total Score,Work/Life Balance Score,Culture & Values Score,Career Opportunities Score,Compensation and Benefits Score,Senior Management Score,Current/Former Employee,Employment Length,Review Title,Review Date,Job Description,Recommend Level,CEO Approval Level,Business Outlook Level,Pros,Cons,Helpful Rate
0,AES Corporation,3,2.0,2.0,4.0,3.0,2.0,Former Employee,more than 10 years,Plus and minus,"Jan 17, 2017",NaN,low,low,middle,Offered incredible opportunity to develop a wi...,Corporate office has a miserable work environm...,6
1,AES Corporation,4,4.0,4.0,4.0,4.0,4.0,Former Employee,NaN,Good place to work,"Jan 24, 2017",Anonymous Employee,high,high,high,"Great values, strong continuous improvement pr...","Frequent cost optimization and restructuring, ...",0
2,AES Corporation,3,3.0,3.0,2.0,2.0,3.0,Current Employee,more than 1 year,Feedback,"Feb 9, 2017",Director in New Delhi,high,middle,middle,Encouragement by Senior Leadership team Sharin...,Salary not aligned to attract & retain best ta...,0
3,AES Corporation,5,5.0,5.0,5.0,5.0,5.0,Current Employee,more than 10 years,Chemistry,"Feb 10, 2017",Analytical Chemist,high,high,high,Is the best places to work.,I haven't any cons to AES.,0
4,AES Corporation,5,4.0,4.0,5.0,4.0,4.0,Current Employee,NaN,Great place to invest time and effort,"Feb 10, 2017","Anonymous Employee in Arlington, VA",high,high,high,The company is focused on accelerating the tra...,Recent growth projects in renewables and clean...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544906,Xylem,3,5.0,5.0,3.0,4.0,1.0,Former Employee,more than 3 years,Immediately Available,"Nov 18, 2019",Senior Software Test Engineer in Bangalore,low,high,high,Work-life balance New technologies in use but ...,Management is not visionary about organization,0
544907,Xylem,3,4.0,4.0,4.0,5.0,2.0,Former Employee,more than 3 years,Great company,"Nov 20, 2019","Field Service Technician II in Charlotte, NC",high,low,low,Benefits are very good compared to others,Going through too many transitions,0
544908,Xylem,4,4.0,4.0,4.0,4.0,3.0,Current Employee,more than 1 year,Decent place to work,"Dec 9, 2019","Lean Six Sigma Black Belt in Nottingham, England",NaN,middle,middle,Good remuneration & Flexible working,"Outdated ERP systems, inefficient processes",2
544909,Xylem,1,3.0,3.0,1.0,2.0,3.0,Former Employee,more than 1 year,Poor management,"Dec 14, 2019","EHS Coordinator in San Diego, CA",NaN,high,low,Hours can be flexible depending on manager,Xylem Inc purchases many small companies cuts ...,1


There are `544,911 * 19` =  **10,353,309** objects in the data.

In [ ]:
def combine_company_data_text(results_df, text_columns):
    # Extract the company names from the results DataFrame
    company_names = results_df["Company Name"]

    # Create an instance of TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Convert the text columns to a dense matrix using TF-IDF vectorization
    arr = vectorizer.fit_transform(results_df.loc[:, text_columns].astype('U')).todense()

    # Create a DataFrame from the dense matrix, using the feature names as column names
    df = pd.DataFrame(arr, columns=vectorizer.get_feature_names_out())

    # Select only the columns that contain alphabetic characters
    columns = [col for col in df.columns if col.isalpha()]
    df = df[columns]

    # Insert the "Company Name" column at the beginning of the DataFrame
    df.insert(0, "Company Name", company_names)

    # Return the resulting DataFrame
    return df


In [ ]:
def combine_company_data_categorical(results_df, categorical_columns, encoder_type="ordinal"):
    # Extract the company names from the results DataFrame
    company_names = results_df["Company Name"]

    if encoder_type == "ordinal":
        # Create an instance of OrdinalEncoder
        encoder = OrdinalEncoder()

        # Encode the categorical columns using ordinal encoding
        arr = encoder.fit_transform(results_df.loc[:, categorical_columns])

        # Use the original column names as the encoded column names
        columns = categorical_columns
    elif encoder_type == "one_hot":
        # Create an instance of OneHotEncoder
        encoder = OneHotEncoder(handle_unknown='ignore')

        # Encode the categorical columns using one-hot encoding and convert to a dense matrix
        arr = encoder.fit_transform(results_df.loc[:, categorical_columns]).todense()

        # Extract the column names from the encoder categories
        columns = [col for col_list in encoder.categories_ for col in col_list]
    else:
        raise ValueError(f"encoder_type must be 'ordinal' or 'one_hot', not {encoder_type}")

    # Create a DataFrame from the encoded matrix, using the column names
    df = pd.DataFrame(arr, columns=columns)

    # Insert the "Company Name" column at the beginning of the DataFrame
    df.insert(0, "Company Name", company_names)

    # Return the resulting DataFrame
    return df


In [ ]:
def apply_pca(results_df, num_dim_pca_text=2):
    # Extract the company names from the results DataFrame
    company_names = results_df["Company Name"]

    # Initialize the PCA object with the specified number of dimensions
    pca = PCA(n_components=num_dim_pca_text)

    # Get the columns to be used for PCA (excluding the "Company Name" column)
    columns = [col for col in results_df.columns if col != "Company Name"]

    # Apply PCA to the selected columns
    arr = pca.fit_transform(results_df[columns])

    # Create new column names for the PCA dimensions
    pca_columns = [f"pc{i+1}" for i in range(num_dim_pca_text)]

    # Create a new DataFrame with the PCA results, including the company names
    df = pd.DataFrame(arr, columns=pca_columns)
    df.insert(0, "Company Name", company_names)

    # Return the resulting DataFrame
    return df


In [ ]:
def combine_company_data(csv_dir, numeric_columns=[], categorical_columns=[], text_columns=[], mean_per_company=False, take_n_random_comments=None, encoder_type="ordinal", num_dim_pca_text=3, drop_columns=None, na_action=None):
    # Retrieve the company data from CSV files and preprocess it
    results_df = get_company_data(csv_dir, take_n_random_comments=take_n_random_comments, drop_columns=drop_columns, na_action=na_action)

    # Initialize the result DataFrame
    df = None

    # Combine categorical columns if specified
    if len(categorical_columns) > 0:
        # Generate the categorical DataFrame using the specified encoder
        categorical_df = combine_company_data_categorical(results_df, categorical_columns, encoder_type=encoder_type)

        # Assign the categorical DataFrame to the result DataFrame
        if df is None:
            df = categorical_df

    # Combine numeric columns if specified
    if len(numeric_columns) > 0:
        # Generate the numeric DataFrame
        numeric_df = combine_company_data_numerical(results_df, numeric_columns)

        # Merge or assign the numeric DataFrame to the result DataFrame
        if df is None:
            df = numeric_df
        else:
            df = df.merge(numeric_df, left_index=True, right_index=True)
            df.insert(0, "Company Name", df["Company Name_x"])
            df = df.drop(["Company Name_x", "Company Name_y"], axis=1)

    # Combine text columns if specified
    if len(text_columns) > 0:
        # Generate the text DataFrame and apply PCA
        text_df = combine_company_data_text(results_df, text_columns)
        pca_text_df = apply_pca(text_df, num_dim_pca_text=num_dim_pca_text)

        # Merge or assign the PCA-transformed text DataFrame to the result DataFrame
        if df is None:
            df = pca_text_df
        else:
            df = df.merge(pca_text_df, left_index=True, right_index=True)
            df.insert(0, "Company Name", df["Company Name_x"])
            df = df.drop(["Company Name_x", "Company Name_y"], axis=1)

    # Return the resulting DataFrame
    return df


Let's take **4,840** objects from the data.

In [ ]:
# Define the directory path where the reviews data is located
REVIEWS_DATA_DIR = "/content/drive/MyDrive/glassdoor_research/Scraping/data/reviews_clean"

# Combine company data based on specified parameters
features_df = combine_company_data(
    REVIEWS_DATA_DIR,  # Directory path where the reviews data is located

    # List of numeric column names to include in the feature DataFrame
    numeric_columns=X_NUMERIC_COLUMNS,

    # List of categorical column names to include in the feature DataFrame
    categorical_columns=X_CATEGORICAL_COLUMNS,

    # List of text column names to include in the feature DataFrame
    text_columns=X_TEXT_COLUMNS,

    mean_per_company=False,  # Indicates whether to average feature values per company
    take_n_random_comments=10,  # Number of random comments to consider per company
    encoder_type="ordinal",  # Type of encoding to use for categorical columns
    num_dim_pca_text=3,  # Number of dimensions to reduce the text columns to using PCA
    drop_columns=DROP_COLUMNS,  # List of column names to drop from the feature DataFrame
    na_action="replace"  # Action to perform when encountering missing values
)

# Display the resulting feature DataFrame
features_df


  0%|          | 0/492 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Corteva 1270 13
1270    NaN
1271    NaN
1272    NaN
1273    NaN
1274    NaN
1275    NaN
1276    NaN
1277    NaN
1278    NaN
1279    NaN
Name: CEO Approval Level, dtype: object


,Company Name,Current/Former Employee,Employment Length,Recommend Level,CEO Approval Level,Business Outlook Level,Total Score,Work/Life Balance Score,Culture & Values Score,Career Opportunities Score,Compensation and Benefits Score,Senior Management Score,Helpful Rate,pc1,pc2,pc3
0,AES Corporation,0.0,0.0,0.0,0.0,0.0,5,5.0,5.0,5.0,5.0,5.0,0,-0.088011,0.084341,-0.026561
1,AES Corporation,1.0,5.0,0.0,0.0,0.0,4,4.0,3.0,4.0,5.0,5.0,0,0.164994,0.047183,-0.032184
2,AES Corporation,1.0,4.0,0.0,0.0,2.0,4,3.0,3.0,2.0,4.0,3.0,1,0.094285,0.008943,-0.036668
3,AES Corporation,0.0,2.0,0.0,0.0,0.0,5,3.0,3.0,5.0,5.0,5.0,0,-0.007977,-0.168400,-0.019108
4,AES Corporation,0.0,4.0,0.0,0.0,0.0,5,3.0,3.0,4.0,5.0,4.0,0,-0.092250,-0.032275,-0.019470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,Xylem,1.0,1.0,1.0,2.0,2.0,2,2.0,2.0,4.0,4.0,2.0,0,-0.046062,-0.052077,-0.014533
4836,Xylem,0.0,0.0,1.0,0.0,1.0,2,5.0,5.0,3.0,1.0,3.0,11,0.066921,-0.051539,-0.008302
4837,Xylem,1.0,5.0,1.0,1.0,0.0,1,2.0,2.0,1.0,1.0,1.0,0,-0.099985,-0.023818,0.007169
4838,Xylem,0.0,3.0,0.0,0.0,0.0,5,5.0,5.0,4.0,4.0,3.0,2,0.096267,0.007138,-0.003596


In [ ]:
def choose_year_as_target(csv_path, year, target_col_name):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path)

    # Extract the year from the "Date" column and create a new "year" column
    df["year"] = df["Date"].apply(lambda x: int(x.split("-")[0]))

    # Filter the DataFrame to include only rows with the specified year
    df = df[df["year"] == year]

    # Select the "Company Name" and target column for the chosen year
    df = df[["Company Name", target_col_name]]

    # Return the resulting DataFrame
    return df


In [ ]:
FINANCIAL_DATA_PATH = "/content/drive/MyDrive/glassdoor_research/Scraping/data/SP500_2020_financialmodelingprep_data_ver3.csv"

# Choose the target year and column
target_df = choose_year_as_target(FINANCIAL_DATA_PATH, 2020, Y_COLUMN)

# Display the resulting DataFrame
target_df


,Company Name,Tobin's Q
2,3M,2.146057
10,A. O. Smith,2.808038
18,Abbott Laboratories,2.689710
26,AbbVie,1.256714
34,Abiomed,12.058538
...,...,...
3711,Yum! Brands,5.583828
3719,Zebra Technologies,3.824299
3727,Zimmer Biomet,1.312602
3735,Zions Bancorporation,0.087196


## EDA

In [ ]:
def pie_plot(features_df, target_df, categorical_columns):
    # Merge features_df and target_df on "Company Name"
    df = features_df.merge(target_df, on="Company Name")

    # Iterate over each categorical column
    for cat_col in categorical_columns:
        # Count the occurrences of each category in the column
        count_df = df.groupby(cat_col).size().to_frame().reset_index()
        count_df = count_df.rename({0: "count"}, axis=1)

        # Create a pie plot using plotly express
        fig = px.pie(count_df, values="count", names=cat_col, title=cat_col)
        fig.show()


In [ ]:
pie_plot(results_df, target_df, X_CATEGORICAL_COLUMNS)

In [ ]:
def scatter_plot(features_df, target_df, target_col_name, dim=2):
    # Merge features_df and target_df based on "Company Name"
    df = features_df.merge(target_df, on="Company Name")

    if dim == 2:
        # Create a scatter plot with two dimensions
        fig = px.scatter(df, x='pc1', y='pc2', size=target_col_name, color=target_col_name, hover_name="Company Name")
        fig.show()
    else:
        # Create a 3D scatter plot with three dimensions
        fig = px.scatter_3d(df, x='pc1', y='pc2', z='pc3', size=target_col_name, color=target_col_name, hover_data=['Company Name'])
        fig.show()


In [ ]:
scatter_plot(features_df, target_df, Y_COLUMN, dim=2)

In [ ]:
#@title
scatter_plot(features_df, target_df, Y_COLUMN, dim=3)

In [ ]:
def correlation_matrix(features_df, target_df, target_col_name):
    # Merge features_df and target_df based on "Company Name" and drop the target_col_name column
    df = features_df.merge(target_df, on="Company Name").drop(target_col_name, axis=1)

    # Compute the correlation matrix
    corr_df = df.corr(numeric_only=True)

    # Create a heatmap plot of the correlation matrix
    fig = px.imshow(corr_df, text_auto=True, aspect="auto")
    fig.show()


In [ ]:
correlation_matrix(features_df, target_df, Y_COLUMN)

## Train

In [ ]:
def train(model, features_df, target_df, target_col_name, scale=True, use_cv=False):
    # Merge features_df and target_df based on "Company Name"
    df = features_df.merge(target_df, on="Company Name")

    # Convert column names to string type
    df.columns = df.columns.astype(str)

    # Extract the feature columns and the target column
    features_columns = [col for col in df.columns if col != "Company Name" and col != target_col_name]
    X = df[features_columns]
    y = df[target_col_name]

    # Create a pipeline with optional feature scaling
    if scale:
        pipe = make_pipeline(StandardScaler(), model)
    else:
        pipe = model

    # Fit the model and compute the R2 score
    if not use_cv:
        pipe.fit(X, y)
        score = pipe.score(X, y)
    else:
        # Use cross-validation if specified
        score = statistics.mean(cross_val_score(pipe, X, y, cv=5))

    # Print the R2 score
    print(f"R2 score: {score}")


In [ ]:
train(LinearRegression(), features_df, target_df, Y_COLUMN)

R2 score: 0.02408253371372182


In [ ]:
train(KNeighborsRegressor(), features_df, target_df, Y_COLUMN)

R2 score: 0.2218384339722509


In [ ]:
train(RandomForestRegressor(),features_df, target_df, Y_COLUMN)

R2 score: 0.8539848793916472


The best [R2 score](https://en.wikipedia.org/wiki/Coefficient_of_determination) is **0.85** by the Random Forest model.

In [ ]:
!jupyter nbconvert --to html /content/YooshfGlassdoor.ipynb

[NbConvertApp] WARNING | pattern '/content/YooshfGlassdoor.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
